In [ ]:
# Materials to Composition String Mapping and Visualization
# ---------------------------------------------------------
# This notebook parses a CSV of material compositions,
# registers them with DeepDRR, and plots attenuation coefficients.

import os
import pandas as pd
import matplotlib.pyplot as plt
from deepdrr.material import Material

# --- Load Materials CSV ---

script_dir = os.getcwd()  # Or use os.path.dirname(__file__) if run as script
csv_path = os.path.join(script_dir, "data/materials.csv")
materials = pd.read_csv(csv_path)
materials.head()

Next we register custom Materials to be used in DeepDRR. After initializing them and registering them to the labelname you want to use they are available in any simulation.

In [ ]:
# --- Build Mapping: material name → composition string (e.g., H0.112O0.888) ---

# Drop mass density column (not part of element composition)
materials = materials.drop(columns=["MassDensity"])

# Dictionary to hold: name → composition string
composition_map = {}

for _, row in materials.iterrows():
    name = row["Name"]
    composition = "".join(
        f"{col}{val:.6f}".rstrip("0").rstrip(".")  # clean float formatting
        for col, val in row.items()
        if col != "Name" and val > 0.0
    )
    composition_map[name] = composition

    # NOTE Register material for DeepDRR
    Material.from_string(composition, compound_string=True)
    Material.register_map({name: composition})

# Preview result
composition_map

In [ ]:
# --- Plot attenuation curves for selected materials ---

def plot_materials(keys):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    for key in keys:
        mat = Material.from_string(key)
        axs[0].plot(mat.energy, mat.mu_over_rho, label=key)
        axs[1].plot(mat.energy, mat.mu_en_over_rho, label=key)

    for ax, ylabel, title in zip(
        axs,
        ["Mass Attenuation Coefficient [cm²/g]", "Mass Energy-Absorption Coefficient [cm²/g]"],
        ["Mass Attenuation Coefficient vs Energy", "Mass Energy-Absorption Coefficient vs Energy"]
    ):
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.set_xlabel("Energy [MeV]")
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.legend()

    plt.tight_layout()
    output_dir = os.path.join(script_dir, "output")
    os.makedirs(output_dir, exist_ok=True)
    plt.show()

# Plot groups
plot_materials(["bone", "bone_0", "bone_1", "bone_2", "bone_3", "bone_4"])
plot_materials(["tissue_soft", "soft_0", "soft_1", "soft_2", "soft_3", "soft_4"])
plot_materials(["air", "air_0", "air_1", "air_2", "air_3", "air_4"])
plot_materials(["blood", "water"])

In [ ]:
# Usage in DeepDRR

from deepdrr import Volume

# Map paths of binary segmentations to material names
file_map = {
    "path/to/bone_binary_segmentation": "bone_0",
    "path/to/tissue_binary_segmentation": "soft_0",
    "path/to/air_binary_segmentation": "air_0",
    "path/to/blood_binary_segmentation": "blood_0",
    "path/to/water_binary_segmentation": "water_0",
}

# TODO maybe this doesnt work because the casting to true false binaries is not working correctly from loading ie different data types
ct_subclustering = Volume.from_nifti("path/to/CT", materials=file_map)